# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install dicom_numpy

In [ ]:
import numpy as np
from scipy import ndimage
from scipy import signal
from skimage.metrics import structural_similarity as compare_ssim
from skimage.transform import resize
import pdb

import pydicom as dicom
import nibabel as nib
import matplotlib.pyplot as plt
import torch
from glob import glob
# import SimpleITK as sitk
import dicom_numpy as dcm2np
import os, re
from skimage.transform import resize
import argparse
from scipy.io import savemat
import time

import cv2
from scipy import ndimage

#Using hd5
import h5py

import scipy.io as sio
import numpy as np
import h5py
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

import argparse
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import torch.optim as optim

import gc, copy
from collections import defaultdict
from torchsummary import summary

import nibabel as nib
from scipy.ndimage import rotate

from sklearn.metrics import mean_squared_error

import sys, h5py

import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
def show_img(img, cmap = 'gray'):
  if img.shape == (256, 256):
    plt.imshow(img, cmap = cmap)
  elif img.shape == (1, 256, 256):
    plt.imshow(np.squeeze(img), cmap = cmap)
  else:
    print("bad shape")

  plt.colorbar()

In [ ]:
def make_figure(img, cmap = 'gray'):
  if img.shape == (256, 256):
    plt.imshow(img, cmap = cmap)
  elif img.shape == (1, 256, 256):
    plt.imshow(np.squeeze(img), cmap = cmap)
  else:
    print("bad shape")

  plt.axis("off")

# SSIM Functions

In [ ]:
def calc_ssim(img1, img2):
    
    K = [0.01, 0.03]
    L = 1
    kernelX = cv2.getGaussianKernel(11, 1.5)
    window = kernelX * kernelX.T
     
    M,N = np.shape(img1)

    C1 = (K[0]*L)**2
    C2 = (K[1]*L)**2
    img1 = np.float64(img1)
    img2 = np.float64(img2)
 
    mu1 = signal.convolve2d(img1, window, 'valid')
    mu2 = signal.convolve2d(img2, window, 'valid')
    
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    
    
    sigma1_sq = signal.convolve2d(img1*img1, window, 'valid') - mu1_sq
    sigma2_sq = signal.convolve2d(img2*img2, window, 'valid') - mu2_sq
    sigma12 = signal.convolve2d(img1*img2, window, 'valid') - mu1_mu2
   
    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    ssim_map = resize(ssim_map, img1.shape)

    mssim = np.mean(ssim_map)
    return ssim_map, mssim

#Gets all the image slices for a CT scan and returns slice-wise scores
def get_SSIM(fd_images, ld_images):
    assert fd_images.ndim == 3
    assert ld_images.ndim == 3
    
    ssim_scores, maps = [], []
    
    #print(ld_images.shape)
    
    for idx in range(ld_images.shape[0]):
        #print(idx)
        img1 = fd_images[idx]
        img2 = ld_images[idx]
        
        '''Calculate the SSIM score'''
        map, score = calc_ssim(img1, img2)
        
        #print(score, gscore)
        ssim_scores.append(score)
        maps.append(map)

    return maps, ssim_scores

# Data Prep

In [ ]:
def sort_dicom(pathdir):
    #pathdir = './LDCT/'
    fd_folds, ld_folds = [], []

    for root, dirs, files in os.walk(data_dir):
        for dir in dirs:
            fold = os.path.join(root,dir+'/')
            if re.search("Full", fold):
                fd_folds.append(fold)
            if re.search("Low", fold+'/'):
                ld_folds.append(fold)
            
    assert len(fd_folds) == len(ld_folds)
    
    #sort the list of dicom files
    fd_folds.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    ld_folds.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

    return fd_folds, ld_folds
#Returns numpy array and image affine from a list of dicoms
def extract_voxel_data(dicom_dir):
    nd_data = np.zeros((0, 512, 512))
    
    for dcm_dir in dicom_dir:
        dcm_files = glob(dcm_dir+'*.dcm')
        dcm_files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[0-9]|[0-9]+', var)])

        datasets = [dicom.read_file(f) for f in dcm_files]
        nd_array, afn = dcm2np.combine_slices(datasets)
    
        nd_array = nd_array.transpose(2, 1, 0)
        nd_data = np.concatenate([nd_data, nd_array], axis=0)
    
    return nd_data 
def window_level(x, w, c, ymin=0, ymax=1.):
    sh = x.shape

    y = np.zeros(sh) #window-leveled image
    
    #print(x[x >= -110 & x < 189])
    #if (x <= c - 0.5 - (w-1) /2), then y = ymin
    mask1 = ( x <= (c - 0.5 - (w-1) /2) )
    y[mask1] = ymin #Update
    
    #else if (x > c - 0.5 + (w-1) /2), then y = ymax
    mask2 = ( x > (c - 0.5 + (w-1) /2) )
    y[mask2] = ymax #Update

    #else y = ((x - (c - 0.5)) / (w-1) + 0.5) * (ymax- ymin) + ymin
    mask3 = ( x > (c - 0.5 - (w-1) /2)) & (x <= (c - 0.5 + (w-1) /2) ) 
    np.putmask(y, mask3, ((x - (c - 0.5)) / (w-1) + 0.5) * (ymax- ymin) + ymin) #Update
    
    return y #return window-leveled image
def prep_slices(volume, scale=256):
    resized_data = []
    
    for i in range(volume.shape[0]):
        '''abdomen: window width=300, center=40'''
        img = window_level(volume[i], 300, 40)
        #print('Img max-min: ', np.max(img), np.min(img))
        
        img = resize(img, [scale, scale])
        assert img.shape == (scale, scale)
        try:
            assert np.max(img) == 1.
            assert np.min(img) == 0.
        except:
            pass
        resized_data.append(img)
    
    resized_data = np.reshape(resized_data, [-1, scale, scale])

    return resized_data
#Simulate low-dose
'''
Given: quarter dose and full dose CT images, required Dose level.
Return: Images at the Dose level
'''
def simulate_ld(I_qd, I_fd, Dose=1.):
    if Dose==1:
        return I_fd
    elif Dose==0.25:
        return I_qd
    else:
        a = np.sqrt(((1/Dose)-1)/3)
        print(a)

        I_noise = I_qd - I_fd

        return I_fd+(a*I_noise)
#normalize input images: high pass filtering
def normalize_image(img):
    gblurr = cv2.GaussianBlur(img,(5,5), cv2.BORDER_DEFAULT) #gaussian blurr
    ds = ndimage.interpolation.zoom(gblurr,.5) #downsample by 2
    I_bar = ndimage.interpolation.zoom(ds, 2) #upsample by 2

    I_norm =  img - I_bar #normalized image obtained after high-pass filtering

    return I_norm
def prepare_error(ld_data, fd_data, h=256, w=256):
    '''
    Noisy input: High Pass Filtering
    I_norm = I - I_bar: I_bar = UpSample(DownSample2(Gauss_Blur(I)))
    '''

    input_norm = []
    error_map = []
    labels = []

    for i in range(ld_data.shape[0]):
        img = ld_data[i]
        E = np.abs(fd_data[i]-ld_data[i]) #Error-map obtained frp, the difference of fd and ld images
        error_map.append(E)

    error_map = np.reshape(error_map, [-1, h, w])  

    return error_map

def prepare_dose_labels(length, dose):
  return np.full((length), dose)

# Data Loading

In [ ]:
class Dataset(Dataset):
  def __init__(self, x, m, d, transform=None):
    self.images = x
    self.maps = m
    self.doses = d
    self.transform = transform

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    images = self.images[idx]
    maps = self.maps[idx]
    doses = self.doses[idx]

    if self.transform:
      images = self.transform(images)
      maps = self.transform(maps)

    return [images, maps, doses]

In [ ]:
# use the same transformations for train/val in this example
trans = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,)) # imagenet
])

#Load train data
def train_DataLoader(data_filename, batch_size):

    with h5py.File(data_filename, 'r') as f:
        images = f['Images'][()]
        maps = f['I_maps'][()]
        doses = f['Dose_labels'][()].astype('float')#.reshape(-1) 
 
    print('Images: ', images.shape, 'Maps: ', maps.shape, 'Doses: ', doses.shape)
   
    indices = np.arange(images.shape[0]) #input indices

    #x_train-->input images, y_train-->Quality map, s_train-->gscores, d_train-->Error map

    x_train, x_val, y_train, y_val, idx_train, idx_val = train_test_split(images, maps, indices, test_size=0.05, random_state=40)###

    d_val = doses[idx_val]
    d_train = doses[idx_train]
    
    train_set = Dataset(x_train, y_train, d_train, transform = trans)
    val_set = Dataset(x_val, y_val, d_val, transform = trans)

    train_loader = {
        'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
        'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
    }

    return train_loader

#Load test data
def test_DataLoader(data_filename, batch_size, norm=True):
    print('Data from ', data_filename)
    try:
        with h5py.File(data_filename, 'r') as f:
          images = f['Images'][()]
          maps = f['I_maps'][()]
          doses = f['Dose_labels'][()].astype('float').reshape(-1) 
    except:
        print('test data not found......')

    print('Images: ', images.shape, 'Maps: ', maps.shape, 'Doses: ', doses.shape)
    test_set = Dataset(images, maps, doses, transform=trans)

    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

    return test_loader

# Define Model

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight, 1.0, 0.02)
        torch.nn.init.constant(m.bias, 0.0)


def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.InstanceNorm2d(in_channels),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.25)
    )

In [ ]:
class SSPWAM(nn.Module):

    def __init__(self):
        super().__init__()
                
        self.dconv_down1 = double_conv(1, 16)
        self.dconv_down2 = double_conv(16, 32)
        self.dconv_down3 = double_conv(32, 64)
        self.dconv_down4 = double_conv(64, 128)
        self.dconv_down5 = double_conv(128, 256)

        # classification branch   
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))       
        self.fc = nn.Linear(256, 5)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.dconv_up4 = double_conv(256 + 128, 128)
        self.dconv_up3 = double_conv(128 + 64, 64)
        self.dconv_up2 = double_conv(64 + 32, 32)
        self.dconv_up1 = double_conv(32 + 16, 16)
        
        self.conv_last = nn.Conv2d(16, 1, 1)
        self.tanh = nn.Tanh() # testing with tanh
        
        
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        conv4 = self.dconv_down4(x)
        x = self.maxpool(conv4)

        x = self.dconv_down5(x)
        x1 = self.maxpool(x)
        
        # classification branch
        avgpool = self.avgpool(x1)
        avgpool = avgpool.view(avgpool.size(0), -1)
        outC = self.fc(avgpool)

        x = self.upsample(x)        
        x = torch.cat([x, conv4], dim=1)

        x = self.dconv_up4(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)       

        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1) 
        
        x = self.dconv_up1(x)
        out = self.conv_last(x)

        out = self.tanh(out)
        
        return out, outC

# Training

In [ ]:
scan = "abdomen"
model = "iqam"
n_epochs = 15
batch_size = 16
lr = 0.0002
img_size = 256

C,H,W = channels, img_size, img_size

In [ ]:
save_path = model + '/' + scan + '/'
os.makedirs(save_path, exist_ok=True)
train_path = '/content/drive/MyDrive/Research/QualityMap_Ayaan/data/abdomen/train/ssim_data_train_dose_final.hdf5'
test_path = '/content/drive/MyDrive/Research/QualityMap_Ayaan/data/abdomen/test/ssim_data_test_dose_final.hdf5'

In [ ]:
def train(model, dataloader, criterion, optimizer, scheduler, epochs, save_path, alpha = 1.0):
  #Training
  torch.autograd.set_detect_anomaly(True)
  best_wts = copy.deepcopy(model.state_dict())
  best_loss = 1e10
  start_time = time.time()

  for epoch in range(n_epochs):
      gc.collect()
      torch.cuda.empty_cache()
      print('Epoch: ', epoch,'...')
      for phase in ['train', 'val']:
          if phase == 'train':
              model.train()
          else:
              model.eval()
        
          
          dose_losses, map_losses = 0.0, 0.0

          for i, (imgs, maps, dose_labels) in enumerate(dataloader[phase]):
              gc.collect()
              torch.cuda.empty_cache()
              
              # Configure input
              batch_imgs = Variable(imgs.type(Tensor))
              batch_maps = Variable(maps.type(Tensor)) # GSSIM Maps
              dose_labels = Variable(dose_labels.type(Tensor))

              # batch_imgs = batch_imgs.to(device=device, dtype=torch.float)
              # batch_maps = batch_maps.to(device=device, dtype=torch.float)
              dose_labels = dose_labels.to(device=device, dtype=torch.long)

              optimizer.zero_grad()

              with torch.set_grad_enabled(phase == 'train'): 
                  # ---------------------
                  #  Train U-Net
                  # ---------------------

                  # Get outputs
                  pred_maps, pred_dose = model(batch_imgs)

                  dose_labels = torch.argmax(dose_labels, 1)

                  # Map loss
                  map_loss = torch.mean(torch.abs(batch_maps - pred_maps))
                  # Dose Loss
                  dose_loss = criterion(pred_dose, dose_labels)

                  # Total Loss

                  loss = map_loss + alpha * dose_loss

                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              
              #Collect losses    
              dose_losses += float(dose_loss)*imgs.size(0)
              map_losses += float(map_loss)*imgs.size(0)

              if i % 10 == 0:
                print("STEP" + str(i))

          print(phase, ": [Batch %d/%d] [Total loss: %f] [Map loss: %f] [Dose loss: %f]" % (i, len(dataloader[phase]), loss.data.cpu(), map_loss.data.cpu(), dose_loss.data.cpu())) 

          epoch_loss = (loss/len(dataloader[phase]))

          if phase == 'train':
              scheduler.step()

          if phase == 'val':
              if epoch_loss < best_loss:
                  print("saving best model...")
                  best_loss = epoch_loss
                  best_wts = copy.deepcopy(model.state_dict())
                  torch.save(best_wts, save_path + f'.pth')
                  print('Model saved at epoch: ', epoch)

  model.load_state_dict(torch.load(save_path + f'.pth'))

  return model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loss function
criterion = torch.nn.CrossEntropyLoss() #torch.nn.BCELoss()

# Initialize models
model = UNet()

# Optimizers

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=1.) #for step-wise leaning rate scheduler

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

alpha = 0.01

save_path = "model"

if cuda: 
    model.cuda()
    criterion.cuda()

print('Model: ', summary(model, input_size=(1, 256, 256)))

In [ ]:
train_dataloader = train_DataLoader(train_path, batch_size)

print('the data is ok')

In [ ]:
gc.collect()
# train(model, dataloader, criterion, optimizer, scheduler, epochs, save_path, alpha)
model = train(model, train_dataloader, criterion, optimizer, scheduler, n_epochs, save_path, alpha)

In [ ]:
from google.colab import files
files.download(save_path + ".pth")